# Solution — Task 03: Testing ML API

## Setup

In [ ]:
import sys, os
from unittest.mock import MagicMock

FIXTURES = os.path.abspath(os.path.join("..", "fixtures", "input"))
if not os.path.exists(FIXTURES):
    FIXTURES = os.path.abspath(os.path.join("fixtures", "input"))
sys.path.insert(0, FIXTURES)

from fastapi.testclient import TestClient
from sample_fastapi_app import app, ml_models

print("Setup complete.")

## Solution 3.1: Test Health Endpoint

In [ ]:
with TestClient(app) as client:
    resp = client.get("/health")
    assert resp.status_code == 200
    data = resp.json()
    assert data["status"] == "ok"
    assert data["model_loaded"] is True
    print(f"Health response: {data}")

# TEST — Do not modify
with TestClient(app) as client:
    resp = client.get("/health")
    assert resp.status_code == 200
    assert resp.json()["status"] == "ok"
    assert resp.json()["model_loaded"] is True
print("Task 3.1 passed!")

## Solution 3.2: Test Predict Endpoint

In [ ]:
with TestClient(app) as client:
    # Test positive text
    resp = client.post("/predict", json={"text": "This is amazing and great"})
    assert resp.status_code == 200
    data = resp.json()
    assert data["label"] in ("positive", "negative", "neutral")
    assert 0 <= data["score"] <= 1
    assert data["text"] == "This is amazing and great"
    print(f"Prediction: {data}")

    # Test negative text
    resp = client.post("/predict", json={"text": "Terrible and awful"})
    assert resp.status_code == 200
    data = resp.json()
    print(f"Negative prediction: {data}")

# TEST — Do not modify
with TestClient(app) as client:
    resp = client.post("/predict", json={"text": "This is amazing and great"})
    assert resp.status_code == 200
    data = resp.json()
    assert "label" in data
    assert "score" in data
    assert "text" in data
    assert 0 <= data["score"] <= 1
    assert data["text"] == "This is amazing and great"
print("Task 3.2 passed!")

## Solution 3.3: Test Validation Errors

In [ ]:
with TestClient(app) as client:
    # Empty text
    resp = client.post("/predict", json={"text": ""})
    assert resp.status_code == 422
    print(f"Empty text: {resp.status_code}")

    # Missing field
    resp = client.post("/predict", json={})
    assert resp.status_code == 422
    print(f"Missing field: {resp.status_code}")

    # Too long
    resp = client.post("/predict", json={"text": "x" * 5001})
    assert resp.status_code == 422
    print(f"Too long: {resp.status_code}")

# TEST — Do not modify
with TestClient(app) as client:
    assert client.post("/predict", json={"text": ""}).status_code == 422
    assert client.post("/predict", json={}).status_code == 422
    assert client.post("/predict", json={"text": "x" * 5001}).status_code == 422
print("Task 3.3 passed!")

## Solution 3.4: Test Batch Predict

In [ ]:
with TestClient(app) as client:
    texts = ["good product", "terrible service", "it was okay"]
    resp = client.post("/predict/batch", json={"texts": texts})
    assert resp.status_code == 200
    preds = resp.json()["predictions"]
    assert len(preds) == 3
    for pred in preds:
        assert "label" in pred
        assert "score" in pred
        assert "text" in pred
    print(f"Batch predictions: {[p['label'] for p in preds]}")

# TEST — Do not modify
with TestClient(app) as client:
    texts = ["good product", "terrible service", "it was okay"]
    resp = client.post("/predict/batch", json={"texts": texts})
    assert resp.status_code == 200
    preds = resp.json()["predictions"]
    assert len(preds) == 3
    for p in preds:
        assert "label" in p and "score" in p and "text" in p
print("Task 3.4 passed!")

## Solution 3.5: Mock the ML Model

In [ ]:
with TestClient(app) as client:
    mock_model = MagicMock()
    mock_model.predict.return_value = ("positive", 0.99)
    ml_models["sentiment"] = mock_model

    resp = client.post("/predict", json={"text": "any text here"})
    assert resp.status_code == 200
    assert resp.json()["label"] == "positive"
    assert resp.json()["score"] == 0.99
    mock_model.predict.assert_called_once_with("any text here")
    print(f"Mock prediction: {resp.json()}")

# TEST — Do not modify
with TestClient(app) as client:
    mock_model = MagicMock()
    mock_model.predict.return_value = ("positive", 0.99)
    ml_models["sentiment"] = mock_model

    resp = client.post("/predict", json={"text": "any text here"})
    assert resp.status_code == 200
    assert resp.json()["label"] == "positive"
    assert resp.json()["score"] == 0.99
    mock_model.predict.assert_called_once_with("any text here")
print("Task 3.5 passed!")

## Solution 3.6: Test Model Not Loaded

In [ ]:
with TestClient(app) as client:
    ml_models.clear()
    resp = client.post("/predict", json={"text": "hello"})
    assert resp.status_code == 503
    assert "not loaded" in resp.json()["detail"].lower()
    print(f"503 response: {resp.json()}")

# TEST — Do not modify
with TestClient(app) as client:
    ml_models.clear()
    resp = client.post("/predict", json={"text": "hello"})
    assert resp.status_code == 503
    assert "not loaded" in resp.json()["detail"].lower()
print("Task 3.6 passed!")